In [1]:
from IPython.display import Latex
import numpy as np
import tensorflow as tf

# 基本概念
## 梯度下降法和最小二乘法（正规方程）的对比
假设有m个训练例子，n个需要训练的特征值
### 梯度下降法
单次迭代公式：$w:= w+\alpha X(y-X^Tw)$

* 需要选择学习率$\alpha$
* 需要多次迭代
* 在n很大的时候工作的也很好

### 最小二乘法
公式：$w := (XX^T)^{-1}Xy$

* 不需要选择学习率$\alpha$
* 不需要迭代
* 需要计算$(XX^T)^{-1}$
* 当n很大时运行非常慢

$XX^T$不可逆的情况
1. 存在特征线性相关，即有多余的特征
解决方案：删除多余的特征
2. 样本数量小于特征数量
解决方案：删除部分特征，或者使用正则化方法

## 反向传播
将预测结果的偏差传递到各个参数上，根据这些参数对偏差的贡献的大小，相应地承担修改的责任
## one-hot向量
只有一个数为1，其他为0

# 信息论知识
## 熵
在信息论中，熵用来衡量一个随机事件的不确定性
* 熵越高，则随机变量的信息越多
* 熵越低，则随机变量的信息越少

**自信息**：一个随机事件所包含的信息量

对于一个随机变量X，当X=x时的自信息I(x)定义为
$$I(x)=-\log p(x)$$

这个公式非常符合我们的直觉，即一件事发生的概率越低，则它包含的信息量越大

### **熵：随机变量X的自信息的数学期望**

对于分布律p(x)，系统的熵为
$$H(x)=E_X[I(X)]=E_X[-\log p(x)]=-\sum_{x\in X}p(x)\log p(x)$$

性质：概率分布越均衡，熵越大
### 熵编码
在对分布p(y)的符号进行编码时，熵H(p)也是理论上最优的平均编码长度，这种编码方式称为熵编码

## 交叉熵
交叉熵是按照概率分布q的最优编码对真实分布为p的信息进行编码的长度
$$H(p,q)=E_p[-\log q(x)]=-\sum_x p(x)\log q(x)$$

在给定q的情况下，如果p和q越接近，交叉熵越小

如果p和q越远，交叉熵就越大

## KL散度
KL散度使用概率分布q来近似p时所造成的的信息损失量

KL散度是按照概率分布q的最优编码对真实分布为p的信息进行编码，其平均编码长度（即交叉熵）H(p,q)和p的最优平均编码长度（即熵）H(p)之间的差异
$$KL(p,q)=H(p,q)-H(p)=\sum_x p(x)\log\frac{p(x)}{q(x)}$$

### 应用到机器学习
以分类为例
* 真实分布$p_r(y|x)$
* 预测分布$p_\theta(y|x)$

$$KL(p_r(y|x),p_\theta (y|x))=\sum_y p_r(y|x)\log\frac{p_r(y|x)}{p_\theta (y|x)}$$
因为$p_r(y|x)$是确定的，所以以上式子正比于
$$-\sum_y p_r(y|x)\log p_\theta(y|x)$$
$$=-\log p_\theta(y^*|x)$$

# 常用numpy函数
## expand_dims
在指定位置插入新的轴来扩展数组形状

例如原本为一维的2个数据，axis=0，则shape变为(1,2),axis=1则shape变为(2,1)
再例如 原本为 (2,3),axis=0，则shape变为(1,2,3),axis=1则shape变为(2,1,3)

In [2]:
# 测试expand_dims函数
x = np.array([1,2,3])
print(x)
print("x.shape: ",x.shape)
y = np.expand_dims(x,axis=0)
print(y)
print("y.shape: ",y.shape)
print("y[0][1]: ",y[0][1])
y = np.expand_dims(x,axis=1)
print(y)
print("y.shape: ",y.shape)
print("y[1][0]: ",y[1][0])

NameError: name 'np' is not defined

## concatenate
根据指定的维度，对一个元组、列表中的list或者ndarray进行连接


In [8]:
# 例子，一个2*2的数组和一个1*2的数组，在第0维进行拼接，得到一个3*2的数组
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
c = np.concatenate((a, b), axis=0)
print(c)

# 一个2*2的数组和一个2*1的数组，在第1维进行拼接，得到一个2*3的数组：
c = np.concatenate((a, b.T), axis=1)
print(c)

[[1 2]
 [3 4]
 [5 6]]
[[1 2 5]
 [3 4 6]]


上面两个简单的例子中，拼接的维度的长度是不同的，但是其他维度的长度必须是相同的，这也是使用concatenate()函数的一个基本原则，违背此规则就会报错，例如一个2\*2的数组和一个1\*2的数组，在第1维进行拼接：

In [ ]:
# 以下代码会报错
np.concatenate((a, b), axis=1)

## stack
将数组的数据按照指定的维度进行堆叠

In [10]:
a = np.array([1,2,3])
b = np.array([2,3,4])
# 按行堆叠
print(np.stack([a,b],axis=0))
# 按列堆叠
print(np.stack([a,b],axis=1))

[[1 2 3]
 [2 3 4]]
[[1 2]
 [2 3]
 [3 4]]


可以看到，进行stack的两个数组必须有相同的形状，同时，输出的结果的维度是比输入的数组都要多一维的。我们拿第一个例子来举例，两个含3个数的一维数组在第0维进行堆叠，其过程等价于先给两个数组增加一个第0维，变为1\*3的数组，再在第0维进行concatenate()操作；第二个例子则是先将两个一维数组变为3*1的二维数组，再在第1维进行concatenate()操作

## vstack
等同于`stack(arrays,axis=0)`
## hstack
等同于`stack(arrays,axis=1)`

## linalg.pinv
求矩阵（通常是非方阵）的伪逆矩阵，X的伪逆矩阵即$(X^TX)^{-1}X^T$（易知$(X^TX)^{-1}X^TX=I$）

# 常用tensorflow函数
## random.uniform
```python
tf.random.uniform(
    shape, minval=0, maxval=None, dtype=tf.dtypes.float32, seed=None, name=None
)
```
生成的值在[minval, maxval) 范围内遵循均匀分布

## reduce_mean
计算张量的各个维度上的元素的平均值
```python
reduce_mean(
    input_tensor,
    axis=None,
    keep_dims=False,
    name=None,
    reduction_indices=None
)
```
axis是tf.reduce_mean函数中的参数,按照函数中axis给定的维度减少input_tensor.除非keep_dims是true,否则张量的秩将在axis的每个条目中减少1
例如

In [2]:
x = tf.constant([[1., 1.], [2., 2.]])
print(tf.reduce_mean(x))  # 1.5
print(tf.reduce_mean(x, 0))  # [1.5, 1.5]
print(tf.reduce_mean(x, 1))  # [1.,  2.]

tf.Tensor(1.5, shape=(), dtype=float32)
tf.Tensor([1.5 1.5], shape=(2,), dtype=float32)
tf.Tensor([1. 2.], shape=(2,), dtype=float32)


# 矩阵微积分
## 标量关于向量的偏导数
$$\frac{\partial y}{\partial \boldsymbol{x}}=[\frac{\partial y}{\partial x_1},\dots,\frac{\partial y}{\partial x_M}]^T$$

## 向量关于向量的偏导数
$$\frac{\partial f(\boldsymbol{x})}{\partial \boldsymbol{x}}=\begin{bmatrix} \frac{\partial y_1}{\partial x_1} & \dots  & \frac{\partial y_N}{\partial x_1} \\
\vdots & \vdots & \vdots \\
 \frac{\partial y_1}{\partial x_M} & \dots & \frac{\partial y_N}{\partial x_M}\end{bmatrix}$$

## 向量函数及其导数
$$\frac{\partial \boldsymbol{x}}{\partial \boldsymbol{x}} = \boldsymbol{I}（单位矩阵）$$
$$\frac{\partial \boldsymbol{Ax}}{\partial \boldsymbol{x}}=\boldsymbol{A}^T $$
$$\frac{\partial \boldsymbol{x^TA}}{\partial \boldsymbol{x}}=\boldsymbol{A} \\$$